In [1]:
# Imports

import os
import shutil

import re
import yaml

import geopandas as gpd
import numpy as np
import rasterio as rio
from rasterio.mask import mask as rio_mask

from vercye_ops.apsim.convert_shapefile_to_geojson import convert_shapefile_to_geojson

In [25]:
# Define parameters and paths
# Please refer to the documentation to understand on how to define the config.

SHAPEFILE_PATH = '/gpfs/data1/cmongp2/sawahnr/data/morocco/OfficialRegions/OfficialShapefiles/admin_boundaries_4.geojson'
ADMIN_COLUMN_NAME = 'ADM4_FR' # Name of the column that contains the administrative division level names for the level of interest in you shapefile.
SHAPEFILE_CENTROID_PROJECTION_CRS = '+proj=aea +lat_1=29 +lat_2=36 +lat_0=32.5 +lon_0=-5 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs'
GEOJSONS_FOLDER = "/gpfs/data1/cmongp2/sawahnr/data/morocco/OfficialRegions/UnpackedRegionsVercye" # Output folder where to save the extracted geojsons

# Optional: Reference raster path - used to get the projection and resolution of the raster for rasterizing the polygons
# Typically should be one of the LAI rasters for the region
# If none, no check if all regions can be rasterized with at least one pixel will be performed
REFERENCE_RASTER_PATH = None

# In the beginning you want to start out with a snakemake config file that is completely filled out except for the "regions" field
# This script will help you fill out the "regions" field with the regions extracted from the shapefile
SNAKEFILE_CONFIG = "/gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/taounate_config.yaml" # Config file for the snakemake pipeline
OUTPUT_DIR = '/gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03' # Directory to save the new head_dir structure and files

# TODO allow different apsim templates for different regions/timepoints?
APSIM_TEMPLATE_PATH = '/gpfs/data1/cmongp2/sawahnr/data/morocco/APSIM_Templates/Morocco_Wheat_18_3_25.apsimx'

In [11]:
# Extracts geojsons in with their corresponding vercye-style directories from a shapefile
# The geojsons are used as a base for the yieldstudy regions

convert_shapefile_to_geojson(shp_fpath=SHAPEFILE_PATH, admin_name_col=ADMIN_COLUMN_NAME, projection_crs=SHAPEFILE_CENTROID_PROJECTION_CRS, output_head_dir=GEOJSONS_FOLDER)

WARNING  Ensure all geometries are at the same administrative level! Use the prepare_shapefile.py script to standardize the shapefile if this is not the case.
WARNING  Processed 118 regions. Validate the output directory /gpfs/data1/cmongp2/sawahnr/data/morocco/OfficialRegions/UnpackedRegionsVercye to check the GeoJSON files are from the correct geoemtries.


In [26]:
# Prints the names of the regions that have at least one pixel after rasterization
# Copy the names of the regions that you want to keep from the output into your snakemake config under regions

# Set these, if you want to filter the regions. 
# E.g you could set filter_col_name = 'ADMIN_2' and filter_col_name = 'Arizona' to only print districts in Arizona.
filter_col_name = 'ADM2_FR'
filter_col_value = 'Taounate'
for f in sorted(os.listdir(GEOJSONS_FOLDER)):
    geojson_folder_path = os.path.join(GEOJSONS_FOLDER, f)
    if not os.path.isdir(geojson_folder_path):
        continue
    
    gdf = gpd.read_file(os.path.join(geojson_folder_path, f + ".geojson"))

    # If user want only a polygons from a specific region, filter
    if filter_col_name is not None and not gdf[filter_col_name][0] == filter_col_value:
        continue

    if REFERENCE_RASTER_PATH is None:
        print(f"- '{f}'")
        continue

    # Identify number of pixels in rasterized shapefile
    src = rio.open(REFERENCE_RASTER_PATH)
    masked_src, masked_transform = rio_mask(src, gdf.geometry, crop=False, nodata=0, indexes=1)
    num_pixels_rasterized = np.count_nonzero(masked_src)

    # Prints region name if at least one pixel is rasterized
    if num_pixels_rasterized != 0:
        print(f"- '{f}'")


- 'ain_aicha'
- 'ain_aicha__centre_'
- 'ain_legdah'
- 'ain_maatouf'
- 'ain_mediouna'
- 'bni_oulid'
- 'bni_ounjel_tafraout'
- 'bni_snous'
- 'bouadel'
- 'bouarouss'
- 'bouchabel'
- 'bouhouda'
- 'el_bibane'
- 'el_bsabsa'
- 'fennassa_bab_el_hit'
- 'galaz'
- 'ghafsai'
- 'jbabra'
- 'karia_ba_mohamed'
- 'khlalfa'
- 'kissane'
- 'loulja'
- 'messassa'
- 'mezraoua'
- 'mkansa'
- 'moulay_abdelkrim'
- 'moulay_bouchta'
- 'oudka'
- 'oued_jemaa'
- 'oulad_ayyad'
- 'oulad_daoud'
- 'ourtzagh'
- 'outabouabane'
- 'ras_el_oued'
- 'ratba'
- 'rghioua'
- 'rhouazi'
- 'sidi_el_abed'
- 'sidi_haj_mhamed'
- 'sidi_mhamed_ben_lahcen'
- 'sidi_mokhfi'
- 'sidi_yahya_bni_zeroual'
- 'tabouda'
- 'tafrant'
- 'tamedit'
- 'taounate'
- 'thar_es-souk'
- 'timezgana'
- 'tissa'
- 'zrizer'


In [27]:
# Creates Folder Structure from Config for years/timepoints, copying all regions
# Ensure you have filled in your config completely before running this script

config = None
with open(SNAKEFILE_CONFIG) as snakemake_config_reader:
    try:
        config = yaml.safe_load(snakemake_config_reader)
    except yaml.YAMLError as e:
        print(e)

years = config['years']
timepoints = config['timepoints']
regions_names = config['regions']

for region_name in regions_names:
    region_file_path = os.path.join(GEOJSONS_FOLDER, region_name, f'{region_name}.geojson')
    
    for year in years:
        year_folder = os.path.join(OUTPUT_DIR, str(year))
        
        for timepoint in timepoints:
            timepoint_folder = os.path.join(year_folder, str(timepoint))
            
            roi_folder = os.path.join(timepoint_folder, region_name)
            os.makedirs(roi_folder, exist_ok=True)

            shutil.copy(region_file_path, roi_folder)

In [28]:
# Copies and adjusts APSIM file to each folder. Adjustment applied for start and end dates as defined in snakemake config for the timepoint
# Attention!: This script assumes that the APSIM file is the same for all regions and timepoints (except start/end dates)

config = None
with open(SNAKEFILE_CONFIG) as snakemake_config_reader:
    try:
        config = yaml.safe_load(snakemake_config_reader)
    except yaml.YAMLError as e:
        print(e)

years = config['years']
timepoints = config['timepoints']
regions_names = config['regions']

for year in years:
    year_folder = os.path.join(OUTPUT_DIR, str(year))

    for timepoint in timepoints:
        timepoint_folder = os.path.join(year_folder, str(timepoint))

        for roi in regions_names:  
            roi_folder = os.path.join(timepoint_folder, roi)

            start_date = config['apsim_params']['time_bounds'][year][timepoint]['sim_start_date']
            end_date = config['apsim_params']['time_bounds'][year][timepoint]['sim_end_date']
            

            with open(APSIM_TEMPLATE_PATH, "r", encoding="utf-8") as file:
                data = file.read()

                # Replace "Start" and "End" dates with new values
                data = re.sub(
                    r'"Start":\s*"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"', 
                    f'"Start": "{start_date}T00:00:00"', 
                    data
                )
                data = re.sub(
                    r'"End":\s*"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"', 
                    f'"End": "{end_date}T00:00:00"', 
                    data
                )

                # Write new file
                print(f'Writing new file for {roi} in {timepoint} of {year} to {roi_folder}')
               
                new_apsim_path = os.path.join(roi_folder, f'{roi}_template.apsimx')
                with open(new_apsim_path, "w", encoding="utf-8") as new_file:
                    new_file.write(data)

Writing new file for ain_aicha in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/ain_aicha
Writing new file for ain_aicha__centre_ in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/ain_aicha__centre_
Writing new file for ain_legdah in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/ain_legdah
Writing new file for ain_maatouf in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/ain_maatouf
Writing new file for ain_mediouna in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/ain_mediouna
Writing new file for bni_oulid in T-0 of 2019 to /gpfs/data1/cmongp2/sawahnr/data/morocco/vercye_runs/taounate_era5-chirps-APSIM-18-03/2019/T-0/bni_oulid
Writing new file for bni_ounjel_tafraout in T-

In [ ]:
# Now Place validation data if available in each timepoint (see docs for more info).